In [8]:
import json
import pandas as pd
import requests
import os
import time

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def getApi(address):
  api_key = "SABXUN7T5GX9YQMDW1EDQJRPKCB2HECT8T"
  return 'https://api-ropsten.etherscan.io/api?module=account&action=tokentx&address='+ address + '&startblock=0&endblock=999999999&sort=asc&apikey=' + 'SABXUN7T5GX9YQMDW1EDQJRPKCB2HECT8T'

def getRespond(sender):
  transactions = ''
  while isinstance(transactions,str):
    time.sleep(1)
    transactions = requests.get(getApi(sender), headers=headers)
  return transactions

hash_set = set()
actors = ['0xeca19b1a87442b0c25801b809bf567a6ca87b1da']
balance_dict = {}
res_hash = []
res_sender = []
res_reciver = []
res_amount = []

while len(actors) != 0:
  next_actors = []

  for actor in actors:
    transactions = getRespond(actor)
    transactions = json.loads(transactions.content)['result']

    for trans in transactions:
      if isinstance(trans, str):
        print(transactions)
        print(trans)
        continue
      if trans['tokenSymbol'] != 'BKTC':
        continue
      if trans['hash'] not in hash_set:
        reciever = trans['to']
        amount = int(int(trans['value'])//1e18)
        sender = trans['from']

        hash = trans['hash']
        hash_set.add(trans['hash'])
        next_actors.append(trans['to'])

        res_hash.append(hash)
        res_sender.append(sender)
        res_reciver.append(reciever)
        res_amount.append(amount)

        if sender not in balance_dict:
          balance_dict[sender] = -int(amount)
        else:
          balance_dict[sender] -= int(amount)

        if reciever not in balance_dict:
          balance_dict[reciever] = int(amount)
        else:
          balance_dict[reciever] += int(amount)
  actors = next_actors

trans_df = pd.DataFrame()
trans_df['Tx Hash'] = res_hash
trans_df['From (Address)'] = res_sender
trans_df['To (Address)'] = res_reciver
trans_df['Amount'] = res_amount

balance_df = pd.DataFrame()
add = []
bal = []
for address in balance_dict:
  if balance_dict[address] >= 0:
    add.append(address)
    bal.append(balance_dict[address])
balance_df['Address'] = add
balance_df["Balance"] = bal

In [9]:
trans_df

,Tx Hash,From (Address),To (Address),Amount
0,0x0681f1579070861285f8d535badb7a3ee7a71fc3b590...,0x70462d3278dac620a4d2c7ae674bc46c6916974e,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,20
1,0x905cdb49db7b8804830017e3f3d4a6d80f30c15f2ef8...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0xee889cdf628e138f15538304fd72e2af56cbe4b6,10
2,0xde88b7bbe2d12527a53d3ce4c0f9c980da5683fd1f5e...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0x4b41947e269464408bbd9dce7346a7e07d522b12,5
3,0x455354c462770b8dd8a72952afaa61cd57b99d113436...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0x873ce5c27fe1a679b724e25b5a856282a93b137f,5
4,0x6ce6f2b521c84bd983752c3304ca5ae1a6b0ba54eaa2...,0xee889cdf628e138f15538304fd72e2af56cbe4b6,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,7
5,0x54467620c2b9a81edcaf551f35971aa7d0ff70459a57...,0xee889cdf628e138f15538304fd72e2af56cbe4b6,0x11759e580b444aa1136482929984c76a19ef2794,3
6,0xd7fb8762f19f4745b6abd6d744df21ffd933905180ea...,0x4b41947e269464408bbd9dce7346a7e07d522b12,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,3
7,0x752f80f1255ef57d37721fb02b227133ba2ef09577b9...,0x70462d3278dac620a4d2c7ae674bc46c6916974e,0x873ce5c27fe1a679b724e25b5a856282a93b137f,35
8,0xae60662ef751a8f682c78fdcf3c9ce6de57b6ec59710...,0x873ce5c27fe1a679b724e25b5a856282a93b137f,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,10
9,0x167b71f723d9bab42391a45b0741d271dbe8885893ae...,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,0x635893f9918cbb41cb6c6bde4a87e5959f53dd56,5


In [10]:
balance_df

,Address,Balance
0,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0
1,0xee889cdf628e138f15538304fd72e2af56cbe4b6,0
2,0x4b41947e269464408bbd9dce7346a7e07d522b12,2
3,0x873ce5c27fe1a679b724e25b5a856282a93b137f,30
4,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,2
5,0x11759e580b444aa1136482929984c76a19ef2794,0
6,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,0
7,0x635893f9918cbb41cb6c6bde4a87e5959f53dd56,5
8,0x04bb39f36bbadf4e31aa1df09a2f5a0b76114a62,2
9,0x8d16abfba10c079340ddf2b252f94353cdccfa57,6
